In [2]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from joblib import load
from tqdm import tqdm

# === Yollar
MODEL_DIR = r"C:\Users\enesk\OneDrive\Desktop\büyük veri"
DATA_DIR = os.path.join(MODEL_DIR, "etiketli_yorum_dosyaları")

# === Modelleri yükle
models = {
    "Openness": load(os.path.join(MODEL_DIR, "model_opn.joblib")),
    "Conscientiousness": load(os.path.join(MODEL_DIR, "model_con.joblib")),
    "Extraversion": load(os.path.join(MODEL_DIR, "model_ext.joblib")),
    "Agreeableness": load(os.path.join(MODEL_DIR, "model_agr.joblib")),
    "Neuroticism": load(os.path.join(MODEL_DIR, "model_neu.joblib"))
}

# === Embedding modeli
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# === Dosya listesi
csv_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".csv")]

tum_df = []

for file in tqdm(csv_files):
    path = os.path.join(DATA_DIR, file)
    try:
        df = pd.read_csv(path, encoding="utf-8-sig")
        if "review_text" not in df.columns or "predicted_label" not in df.columns:
            print(f"Eksik sütunlar: {file}")
            continue

        yorumlar = df["review_text"].astype(str).tolist()
        embeddings = embedder.encode(yorumlar, show_progress_bar=False)

        for trait, model in models.items():
            df[trait] = model.predict(embeddings)

        df["oyun"] = file.replace("etiketli_", "").replace("_yorumlar.csv", "").replace("_", " ")
        tum_df.append(df)

    except Exception as e:
        print(f"❌ Hata ({file}): {e}")

# === Hepsini birleştir
birlesik_df = pd.concat(tum_df, ignore_index=True)
birlesik_df.to_csv(os.path.join(MODEL_DIR, "yorum_turu_ve_kisilik_etiketli.csv"), index=False, encoding="utf-8-sig")

print("✅ Birleştirilmiş veri dosyası oluşturuldu.")


c:\Users\enesk\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Ridge from version 1.4.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
100%|██████████| 55/55 [13:32<00:00, 14.77s/it]


✅ Birleştirilmiş veri dosyası oluşturuldu.


In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from joblib import dump

# === 1. Veriyi yükle
df = pd.read_csv("yorum_turu_ve_kisilik_etiketli.csv", encoding="utf-8-sig")
df.dropna(subset=["review_text", "predicted_label"], inplace=True)

# === 2. Embed modeli
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# === 3. Giriş ve etiketleri ayır
X_text = df["review_text"].astype(str).tolist()
y_class = df["predicted_label"]
y_traits = df[["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]]

# === 4. Embed (yorum metinlerini vektöre çevir)
X_embed = embedder.encode(X_text, show_progress_bar=True)

# === 5. Eğitim/test ayrımı
X_train, X_test, y_train_cls, y_test_cls = train_test_split(X_embed, y_class, test_size=0.2, random_state=42)
_, _, y_train_reg, y_test_reg = train_test_split(X_embed, y_traits, test_size=0.2, random_state=42)

# === 6. Sınıflandırıcı model
cls_model = LogisticRegression(max_iter=1000)
cls_model.fit(X_train, y_train_cls)
y_pred_cls = cls_model.predict(X_test)
print("🎯 Yorum Türü Sınıflandırması:\n", classification_report(y_test_cls, y_pred_cls))

# === 7. 5 ayrı kişilik regresyon modeli
reg_models = {}
for trait in y_traits.columns:
    reg = Ridge()
    reg.fit(X_train, y_train_reg[trait])
    y_pred = reg.predict(X_test)
    mse = mean_squared_error(y_test_reg[trait], y_pred)
    reg_models[trait] = reg
    print(f"🧠 {trait} MSE: {mse:.4f}")

# === 8. Modelleri kaydet (.joblib)
dump(cls_model, "model_yorum_turu.joblib")
for trait, model in reg_models.items():
    dump(model, f"model_{trait[:3].lower()}.joblib")

print("✅ Tüm modeller başarıyla eğitildi ve kaydedildi.")

Batches:   0%|          | 0/12108 [00:00<?, ?it/s]

🎯 Yorum Türü Sınıflandırması:
                 precision    recall  f1-score   support

bilgilendirici       0.77      0.83      0.80     34971
     eleştirel       0.70      0.82      0.76     29451
fanboy/fangirl       0.54      0.15      0.24      1282
          nötr       0.50      0.04      0.08      2952
   şaka amaçlı       0.56      0.29      0.38      8831

      accuracy                           0.72     77487
     macro avg       0.61      0.43      0.45     77487
  weighted avg       0.70      0.72      0.70     77487

🧠 Openness MSE: 0.0000
🧠 Conscientiousness MSE: 0.0000
🧠 Extraversion MSE: 0.0000
🧠 Agreeableness MSE: 0.0000
🧠 Neuroticism MSE: 0.0000
✅ Tüm modeller başarıyla eğitildi ve kaydedildi.


In [2]:
import requests
import random
from sentence_transformers import SentenceTransformer
from joblib import load
import numpy as np

# === 1. Model yükleme
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
cls_model = load("model_yorum_turu.joblib")
reg_models = {
    "Openness": load("model_opn.joblib"),
    "Conscientiousness": load("model_con.joblib"),
    "Extraversion": load("model_ext.joblib"),
    "Agreeableness": load("model_agr.joblib"),
    "Neuroticism": load("model_neu.joblib")
}

# === 2. 100 yorumdan rastgele 1 tanesini seç
def steam_yorum_api_rastgele(app_id):
    url = f"https://store.steampowered.com/appreviews/{app_id}"
    params = {
        "json": 1,
        "num_per_page": 100,
        "language": "english",
        "filter": "recent"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        reviews = data.get("reviews", [])
        if reviews:
            review = random.choice(reviews)
            return review["review"]
    return None

# === 3. Tahmin
def tahmin_et(yorum_metni):
    embed = embedder.encode([yorum_metni])
    y_pred = cls_model.predict(embed)[0]
    y_proba = cls_model.predict_proba(embed)[0]
    confidence = np.max(y_proba)

    print(f"\n🎭 Tahmini Tür: {y_pred} ({confidence:.2%} güven)")
    print("🧠 Kişilik Skorları:")
    for trait, model in reg_models.items():
        skor = model.predict(embed)[0]
        print(f"  {trait}: {skor:.3f}")

# === 4. Ana entegrasyon
def yorum_api_ile_tahmin(app_id):
    yorum = steam_yorum_api_rastgele(app_id)
    if yorum:
        print(f"\n📜 Yorum:\n{yorum}\n")
        tahmin_et(yorum)
    else:
        print("⚠️ Yorum çekilemedi.")

# === 5. Örnek çağrı: Hades (1145360)
yorum_api_ile_tahmin(1145360)

c:\Users\enesk\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Ridge from version 1.4.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(



📜 Yorum:
if you love greek myth and unlimited action you would love it !!


🎭 Tahmini Tür: bilgilendirici (77.03% güven)
🧠 Kişilik Skorları:
  Openness: 0.587
  Conscientiousness: 0.551
  Extraversion: 0.432
  Agreeableness: 0.629
  Neuroticism: 0.435
